<a href="https://colab.research.google.com/github/Aryamaan23/-100daysofcode/blob/main/CMD_Anomaly_Updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:






import pandas as pd
import numpy as np

In [ ]:
adr1 = pd.read_excel(r'C:\Users\40101972\OneDrive - Anheuser-Busch InBev\Kumaresh Sadasivan\Anomaly Execution\CMD\Europe\ADR6_1.xlsx')
knb1 = pd.read_excel(r"C:\Users\40101972\OneDrive - Anheuser-Busch InBev\Kumaresh Sadasivan\Anomaly Execution\CMD\Europe\KNB1.xlsx")
knvv = pd.read_excel(r'C:\Users\40101972\OneDrive - Anheuser-Busch InBev\Kumaresh Sadasivan\Anomaly Execution\CMD\Europe\KNVV.xlsx')
zv17 = pd.read_excel(r'C:\Users\40101972\OneDrive - Anheuser-Busch InBev\Kumaresh Sadasivan\Anomaly Execution\CMD\Europe\ZV17.xlsx')
knkk = pd.read_excel(r'C:\Users\40101972\OneDrive - Anheuser-Busch InBev\Kumaresh Sadasivan\Anomaly Execution\CMD\Europe\KNKK.xlsx')
zv17soldto = pd.read_excel(r'C:\Users\40101972\OneDrive - Anheuser-Busch InBev\Kumaresh Sadasivan\Anomaly Execution\CMD\Europe\ZV17_Sold to.xlsx')

In [ ]:
kna1 = pd.read_excel(r'C:\Users\40101972\OneDrive - Anheuser-Busch InBev\Kumaresh Sadasivan\Anomaly Execution\CMD\Europe\KNA1.xlsx')
kna1_tax = pd.read_excel(r'C:\Users\40101972\OneDrive - Anheuser-Busch InBev\Kumaresh Sadasivan\Anomaly Execution\CMD\Europe\kna1_tax.xlsx')
kna1.drop(kna1[kna1['Customer'].isnull()].index,inplace=True)
kna1['Customer'] = kna1['Customer'].apply(int)
zv17.rename(columns={'Linked ship-to':'Customer'},inplace=True)

ValueError: invalid literal for int() with base 10: 'AANL000020'

In [ ]:
kna1.head()

### TAX ANOMALY CHECK

In [ ]:
df_1 = pd.merge(kna1_tax,knvv[['Customer','SOrg.']],on='Customer',how='left')
df_1['Tax Number 1'] = df_1['Tax Number 1'].apply(str) 

tax=[]
for i in range(df_1.shape[0]):
    if ((df_1['Group'][i] == "Z003") and ((df_1['SOrg.'][i] == "BE11") or (df_1['SOrg.'][i] == "IT01"))):
        if df_1['Tax Number 1'][i] == "": 
            tax.append("Anomaly")
        else:
            tax.append("No Anomaly")
    else:
        tax.append("Out of Scope")
df_1['Tax Anomaly Check'] = pd.DataFrame(tax) 

df_1.to_csv(r'C:\Users\40101972\OneDrive - Anheuser-Busch InBev\Kumaresh Sadasivan\Anomaly Execution\CMD\Europe\Output\Revised Logics\tax_anomaly.csv')

In [ ]:
df_1['Tax Anomaly Check'].value_counts()

## PAYMENT TERMS CHECK

In [ ]:
df_2 = pd.merge(zv17,kna1[kna1['Account group'] == "Z002"][['Customer','A5']],on='Customer',how='left')
df_2.drop("Customer",axis=1,inplace=True)
df_2.rename(columns={'Customer Number':'Customer'},inplace=True)
df_2['Customer'] = df_2['Customer'].apply(str)

knb1['Customer'] = knb1['Customer'].apply(str)
knb1['Customer_start'] = knb1['Customer'].apply(lambda x : x[0])
knb1['3series'] = (knb1['Customer_start'] == '3')
knb1['CoCd'] = knb1['CoCd'].apply(str)
df_2['Customer'] = df_2['Customer'].apply(str)

knb1['Country'] = knb1['CoCd'].apply(lambda x : x[:2])

df_payt = pd.merge(knb1[knb1['3series'] == 1][['Customer','Country','PayT']],df_2,how='left',on='Customer')
df_payt.drop_duplicates(keep='first',inplace=True)
df_payt.index = range(df_payt.shape[0])



In [ ]:
df_payt

In [ ]:
payt = []
df_payt['A5'].fillna("nan",inplace=True)
for i in range(df_payt.shape[0]):
    if df_payt['A5'][i] == 'nan':
        payt.append("Attribute 5 value missing")
    elif ((df_payt['Country'][i] == "NL") or (df_payt['Country'][i] == "BE") or (df_payt['Country'][i] == "LU")):
        if df_payt['A5'][i] == 10:
            if df_payt['PayT'][i] == 'nan':
                payt.append("Anomaly")
            else:
                payt.append("Not Anomaly") 
        elif df_payt['A5'][i] == 20:
            if df_payt['PayT'][i] == '000A':
                payt.append("Not Anomaly")
            else:
                payt.append("Anomaly")
        else:
            payt.append("Out of scope")
                
    elif ((df_payt['Country'][i] == "AT") or (df_payt['Country'][i] == "CH") or (df_payt['Country'][i] == "PL") or (df_payt['Country'][i] == "IT")):
        if (df_payt['A5'][i] == 10) or (df_payt['A5'][i] == 20) :
            if df_payt['PayT'][i] == 'nan':
                payt.append("Anomaly")
            else:
                payt.append("Not Anomaly")     
        else:
            payt.append("Out of scope")
            
    elif ((df_payt['Country'][i] == "SE") or (df_payt['Country'][i] == "DK") or (df_payt['Country'][i] == "FI") or (df_payt['Country'][i] == "NO")):
        if (df_payt['A5'][i] == 10) or (df_payt['A5'][i] == 20) :
            if (df_payt['PayT'][i] == '015I') or (df_payt['PayT'][i] == '030I'):
                payt.append("Not Anomaly")
            else:
                payt.append("Anomaly") 
        else:
            payt.append("Out of scope")
    
    elif df_payt['Country'][i] == "DE":
        if df_payt['A5'][i] == 20:
            if (df_payt['PayT'][i] == '001I') :
                payt.append("Not Anomaly")
            else:
                payt.append("Anomaly") 
        elif df_payt['A5'][i] == 10:
            if df_payt['PayT'][i] == 'nan':
                payt.append("Anomaly")
            else:
                payt.append("Not Anomaly")
    
    elif df_payt['Country'][i] == "FR":
        if (df_payt['PayT'][i] == "030M") or (df_payt['PayT'][i] == "045M"):
            payt.append("Not Anomaly")
        else:
            payt.append("Out of scope")
        
    elif df_payt['Country'][i] == "GB":
        if df_payt['A5'][i] == 10:
            if df_payt['PayT'][i] == "nan":
                payt.append("Anomaly")
            else:
                payt.append("Not Anomaly")
        else:
            payt.append("Out of scope")
            
df_payt['Payment Terms Check'] = pd.DataFrame(payt)


In [ ]:
df_payt['Payment Terms Check'].value_counts()

In [ ]:
df_payt.to_csv(r'C:\Users\40101972\OneDrive - Anheuser-Busch InBev\Kumaresh Sadasivan\Anomaly Execution\CMD\Europe\Output\Revised Logics\Payment_terms.csv')

### CREDIT LIMIT CHECK

In [ ]:
knkk['Cred.contr.area'] = knkk['Cred.contr.area'].apply(str)
knkk['Country'] = knkk['Cred.contr.area'].apply(lambda x : x[:2])
knkk['Customer'] = knkk['Customer'].apply(str)
knkk['Customer_start'] = knkk['Customer'].apply(lambda x: x[0])
knkk['Credit limit'] = knkk['Credit limit'].apply(str)

df_cred = pd.merge(knkk[knkk['Customer_start'] == "3"][['Customer','Credit limit','Country']],df_2[['Customer','A5']],on='Customer',how='left')
df_cred['A5'].fillna("nan",inplace=True)
df_cred.drop_duplicates(keep='first',inplace=True)
df_cred.index = range(df_cred.shape[0])
df_cred['Credit limit'] = df_cred['Credit limit'].apply(lambda x : x.lstrip())
df_cred['Credit limit'] = df_cred['Credit limit'].apply(lambda x : x.rstrip())

NameError: ignored

In [ ]:
knkk

In [ ]:
cred = []
for i in range(df_cred.shape[0]):
    if df_cred['A5'][i] == 'nan':
        cred.append("Attribute 5 is missing")
        
    elif ((df_cred['Country'][i] == "IT") or (df_cred['Country'][i] == "DK") or (df_cred['Country'][i] == "FI") or (df_cred['Country'][i] == "SE") or (df_cred['Country'][i] == "NO") or (df_cred['Country'][i] == "GB") or (df_cred['Country'][i] == "FR")):
        if df_cred['A5'][i] == 10:
            if df_cred['Credit limit'][i] == 'nan':
                cred.append("Anomaly")
            else:
                cred.append("Not Anomaly")
        elif df_cred['A5'][i] == 20:
            if df_cred['Credit limit'][i] == 'nan':
                cred.append("Anomaly")
            else:
                cred.append("Not Anomaly")
        else:
            cred.append("Out of scope")
        
    elif ((df_cred['Country'][i] == "BE") or (df_cred['Country'][i] == "NL")):
        if (df_cred['A5'][i] == 10):
            if df_cred['Credit limit'][i] == 'nan':
                cred.append("Anomaly")
            else:
                cred.append("Not Anomaly")
        elif (df_cred['A5'][i] == 20):
            if df_cred['Credit limit'][i] == '0,01':
                cred.append("Not Anomaly")
            else:
                cred.append("Anomaly")
        else:
            cred.append("Out of scope")
                
    elif (df_cred['Country'][i] == "DE"):
        if df_cred['A5'][i] == 10:
            if df_cred['Credit limit'][i] == '0,01':
                cred.append("Not Anomaly")
            else:
                cred.append("Anomaly")
                
    elif (df_cred['Country'][i] == "LU"):
        if df_cred['A5'][i] == 5 :
            if df_cred['Credit limit'][i] == '100':
                cred.append("Not Anomaly")
            else:
                cred.append("Anomaly")
        elif df_cred['A5'][i] == 10:
            if df_cred['Credit limit'][i] == '100':
                cred.append("Not Anomaly")
            else:
                cred.append("Anomaly")
                
    elif ((df_cred['Country'][i] == "AT") or (df_cred['Country'][i] == "PL") or (df_cred['Country'][i] == "CH")):
        if df_cred['Credit limit'][i] == "nan":
            cred.append("Anomaly")
        else:
            cred.append("Not Anomaly")
    else:
        cred.append("Out of scope")
                
df_cred['Credit Limit Anomaly Check'] = pd.DataFrame(cred)


In [ ]:
df_cred['Credit Limit Anomaly Check'].value_counts()

In [ ]:
df_cred.to_csv(r'C:\Users\40101972\OneDrive - Anheuser-Busch InBev\Kumaresh Sadasivan\Anomaly Execution\CMD\Europe\Output\Revised Logics\Credit_terms.csv')

### RISK CATEGORY CHECK

In [ ]:
df_risk = pd.merge(knkk[knkk['Customer_start'] == "3"][['Customer','Risk category','Country']],df_2[['Customer','A5']],on='Customer',how='left')
df_risk.fillna("nan",inplace=True)

In [ ]:
risk = []
for i in range(df_risk.shape[0]):
    if df_risk['A5'][i] == 'nan': 
        risk.append('Attribute 5 is missing')
    elif (df_risk['Country'][i] == "NL") or (df_risk['Country'][i] == "BE") or (df_risk['Country'][i] == "AT") or (df_risk['Country'][i] == "CH") or (df_risk['Country'][i] == "PL") or (df_risk['Country'][i] == "DE") or (df_risk['Country'][i] == "IT"):
        if df_risk['A5'][i] == 10:
            if df_risk['Risk category'][i] == 'nan':
                risk.append("Anomaly")
            else:
                risk.append("Not Anomaly")
        elif df_risk['A5'][i] == 20:
            if df_risk['Risk category'][i] == "D":
                risk.append("Not Anomaly")
            else: 
                risk.append('Anomaly')
    elif (df_risk['Country'][i] == "SE") or (df_risk['Country'][i] == "DK") or (df_risk['Country'][i] == "FI") or (df_risk['Country'][i] == "NO"):
        if (df_risk['A5'][i] == 10) or (df_risk['A5'][i] == 20) :
            if df_risk['Risk category'][i] == 'nan':
                risk.append("Anomaly")
            else:
                risk.append("Not Anomaly")
    elif df_risk['Country'][i] == "FR":
        if df_risk['Risk category'] == "D":
            risk.append("Not Anomaly")
        else:
            risk.append("Anomaly")
    elif df_risk['Country'][i] == "LU":
        if df_risk['Risk category'][i] == "F":
            risk.append("Not Anomaly")
        else:
            risk.append("Anomaly")
    elif df_risk['Country'][i] == "GB":
        if df_risk['A5'][i] == 10:
            if df_risk['Risk category'][i] == 'nan':
                risk.append("Anomaly")
            else:
                risk.append("Not Anomaly")
        elif df_risk['A5'][i] == 20:
            if df_risk['Risk category'][i] == "E":
                risk.append("Not Anomaly")
            else: 
                risk.append('Anomaly')
    
        else:
            risk.append("Out of Scope")
df_risk['Anomaly'] = pd.DataFrame(risk)    

In [ ]:
df_risk.head()

In [ ]:
df_risk['Anomaly'].value_counts()

In [ ]:
df_risk.to_csv(r'C:\Users\40101972\OneDrive - Anheuser-Busch InBev\Kumaresh Sadasivan\Anomaly Execution\CMD\Europe\Output\Revised Logics\risk_category.csv')

### ACCOUNTING CLERK CHECK

In [ ]:
df_clerk = knb1[knb1['Customer_start'] == '3'][['Customer','Acctg clerk']]
df_clerk['Acctg clerk'].fillna("nan",inplace=True)
df_clerk['Accounting Clerk Check'] = (df_clerk['Acctg clerk'] != 'nan')
df_clerk.replace(to_replace=[True,False],value=['Not Anomaly','Anomaly'],inplace=True)

In [ ]:
df_clerk.head()

In [ ]:
df_clerk.to_csv(r'C:\Users\40101972\OneDrive - Anheuser-Busch InBev\Kumaresh Sadasivan\Anomaly Execution\CMD\Europe\Output\Revised Logics\accounting_clerk.csv')

### ANOMALY IF NAN, OTHERWISE NO ANOMALY - FOR ACCOUNTING CLERK ANOMALY - knkk 

### Shipping condition anomaly - merge zv17_soldto with kna1,then merge with knvv - Take A5 value

In [ ]:
zv17soldto.rename(columns={"Linked ship-to":"Customer"},inplace=True)
df_ship = pd.merge(zv17soldto[['Customer Number','Customer']],kna1[['Customer','A5']],on='Customer',how='left')

In [ ]:
zv17soldto

In [ ]:
df_ship

In [ ]:
knvv['Customer'] = knvv['Customer'].apply(str)
knvv['Customer_start'] = knvv['Customer'].apply(lambda x : x[0])
knvv['one series'] = (knvv['Customer_start'] == "1")
knvv['len'] = knvv['Customer'].apply(len)
knvv_new = knvv[(knvv['one series'] == 1) & (knvv['len'] == 8)]
df_ship['Customer'] = df_ship['Customer'].apply(str)
x = pd.merge(knvv_new,df_ship,on='Customer',how='left')
x['Country'] = x['SOrg.'].apply(lambda x : x[:2])
x['A5'].fillna('nan',inplace=True)

In [ ]:
ship = []
for i in range(x.shape[0]):
    if x['A5'][i] == 'nan':
        ship.append("Attribute 5 value missing")
    elif (x['Country'][i] == "BE") or (x['Country'][i] == "IT") or (x['Country'][i] == "AT") or (x['Country'][i] == "DE") or (x['Country'][i] == "LU") or (x['Country'][i] == "FR"):
        if x['SC'][i] == "T1":
            ship.append("Not Anomaly")
        else:
            ship.append("Anomaly")
            
    elif (x['Country'][i] == "DK") or (x['Country'][i] == "FI") or (x['Country'][i] == "SE") or (x['Country'][i] == "NO"):
        if x['SC'][i] == "X1":
            ship.append("Not Anomaly")
        else:
            ship.append("Anomaly")
            
    elif x['Country'][i] == "NL":
        if x['A5'][i] == 10:
            if x['SC'][i] == "TJ":
                ship.append("Not Anomaly")
            else:
                ship.append("Anomaly")
        else:
            if x['SC'][i] == "T1":
                ship.append("Not Anomaly")
            else:
                ship.append("Anomaly")
                
    elif x['Country'][i] == "GB":
        if x['A5'][i] == 10:
            if x['SC'][i] == "nan":
                ship.append("Anomaly")
            else:
                ship.append("Not Anomaly")
        else:
            if x['SC'][i] == "T1":
                ship.append("Not Anomaly")
            else:
                ship.append("Anomaly")
        
        
x['Shipping Anomaly'] = pd.DataFrame(ship)
x.drop_duplicates(keep='first',inplace=True)

In [ ]:
x.head()

In [ ]:
x['Shipping Anomaly'].value_counts()

In [ ]:
x.to_csv(r'C:\Users\40101972\OneDrive - Anheuser-Busch InBev\Kumaresh Sadasivan\Anomaly Execution\CMD\Europe\Output\Revised Logics\shipping_condition.csv')